In [1]:
from utils.get_data import Get_Opendata
from utils.tools import fiilna
from utils.hana_connection import Upload_Test_Environment,Upload_Deployment_Environment
from data_preprocessing import Openso_Preprocess, Upload_Data
from Indicator_Construction import Foward_Indicator, Trend_Table
from dateutil.relativedelta import relativedelta
import pandas as pd
import datetime

In [2]:
Open_so_final=Get_Opendata()

d:\SCM 指標\utils\get_data.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  open_so_ori=pd.read_sql(sql_schema_open_so, extract_connection)
d:\SCM 指標\utils\get_data.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  open_so_c_ori=pd.read_sql(sql_schema_open_so_c, extract_connection)


In [36]:
today=datetime.datetime.today()
start_date=(today+relativedelta(months=-2)).strftime('%Y-%m-01')
end_date=(today+relativedelta(months=2)).strftime('%Y-%m-01') #要預測的月
full_date_range = pd.date_range(start=start_date,end=end_date,freq='MS').normalize() 
print('現在時間(年/月):',today.strftime('%Y-%m-01'))
print('預測時間(年/月):',end_date)
print(f'資料時間(年/月): {start_date} to {end_date}')

現在時間(年/月): 2024-11-01
預測時間(年/月): 2025-01-01
資料時間(年/月): 2024-09-01 to 2025-01-01


In [50]:
def main(ship_plant):
    Region_class=ship_plant # CN:China (CKH是HUB廠), TW:Taiwan, EU , US
    ABC_class=('A','B','C') # A or B or C ABC_INDICATOR.str.match("^[ABC]")
    PG_class=['ISG','IAG','ICWG','ESG','ECG', 'IPSG','IDSG'] #IIOT->4['ISG','IAG','ICWG','ESG'] EIOT->3 ['ECG', 'IPSG','IDSG']
    #根據上述條件建立sql
    Constrain_query="ABC_INDICATOR.str.startswith(@ABC_class)  and PG in @PG_class and SHIP_PLANT.str.startswith(@Region_class) and  year_month>=@start_date and year_month<=@end_date"
    Openso_1=Openso_Preprocess(data=Open_so_final,version_year=1).query(Constrain_query).reset_index(drop=True)
    Openso_2=Openso_Preprocess(data=Open_so_final,version_year=2).query(Constrain_query).reset_index(drop=True)
    Openso_3=Openso_Preprocess(data=Open_so_final,version_year=3).query(Constrain_query).reset_index(drop=True)
    # 料號日期條件
    Material_sql=' PART_NO==@PART_NO_name '
    date_column='year_month'
    new_data=pd.DataFrame()
    today_YM=today.strftime('%Y-%m-01')

    for PART_NO_name in Openso_1.PART_NO.unique():
        #open part
        PART_NO_Openso_1=Openso_1.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
        PART_NO_Openso_2=Openso_2.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
        PART_NO_Openso_3=Openso_3.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
        '''
        填補so & openso
        '''
        PART_NO_Openso_1=fiilna(PART_NO_Openso_1, full_date_range , QTY_name= 'OPEN_QTY' )
        PART_NO_Openso_2=fiilna(PART_NO_Openso_2, full_date_range , QTY_name= 'OPEN_QTY' )
        PART_NO_Openso_3=fiilna(PART_NO_Openso_3, full_date_range , QTY_name= 'OPEN_QTY' )

        #計算兩種backlog變化率的計算
        Indicator= Foward_Indicator(PART_NO_Openso_1, PART_NO_Openso_2, PART_NO_Openso_3 )
        #判定條件是否為趨勢
        Trend_Indicator =Trend_Table(Indicator)

        temp=pd.DataFrame()
        temp['MATNR']=[PART_NO_name]
        temp['AI_TREND']=Trend_Indicator.loc[Trend_Indicator['Date'] == today_YM, 'labels'].values
        new_data = pd.concat([new_data, temp], ignore_index=True)

    Commit_data= Upload_Data(new_data,Region_class,today)
    #upload測試機/正式機
    # Upload_Test_Environment(Commit_data)
    # Upload_Deployment_Environment(Commit_data)
    print('地區:', Region_class)
    print("Commit data shape:",Commit_data.shape)

In [51]:
Region=['CN','TW','EU','US']
for ship_plant in Region:
    main(ship_plant)

地區: CN
Commit data shape: (419, 14)
地區: TW
Commit data shape: (474, 14)
地區: EU
Commit data shape: (374, 14)
地區: US
Commit data shape: (264, 14)


In [40]:
Region_class='US' # CN:China (CKH是HUB廠), TW:Taiwan, EU , US
ABC_class=('A','B','C') # A or B or C ABC_INDICATOR.str.match("^[ABC]")
PG_class=['ISG','IAG','ICWG','ESG','ECG', 'IPSG','IDSG'] #IIOT->4['ISG','IAG','ICWG','ESG'] EIOT->3 ['ECG', 'IPSG','IDSG']
#根據上述條件建立sql
Constrain_query="ABC_INDICATOR.str.startswith(@ABC_class)  and PG in @PG_class and SHIP_PLANT.str.startswith(@Region_class) and  year_month>=@start_date and year_month<=@end_date"


In [41]:
Openso_1=Openso_Preprocess(data=Open_so_final,version_year=1).query(Constrain_query).reset_index(drop=True)
Openso_2=Openso_Preprocess(data=Open_so_final,version_year=2).query(Constrain_query).reset_index(drop=True)
Openso_3=Openso_Preprocess(data=Open_so_final,version_year=3).query(Constrain_query).reset_index(drop=True)

In [42]:
# 料號日期條件
Material_sql=' PART_NO==@PART_NO_name '
date_column='year_month'
new_data=pd.DataFrame()
today_YM=today.strftime('%Y-%m-01')

for PART_NO_name in Openso_1.PART_NO.unique():
    #open part
    PART_NO_Openso_1=Openso_1.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    PART_NO_Openso_2=Openso_2.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    PART_NO_Openso_3=Openso_3.query(Material_sql).groupby([date_column])['OPEN_QTY'].sum().reset_index()
    '''
    填補so & openso
    '''
    PART_NO_Openso_1=fiilna(PART_NO_Openso_1, full_date_range , QTY_name= 'OPEN_QTY' )
    PART_NO_Openso_2=fiilna(PART_NO_Openso_2, full_date_range , QTY_name= 'OPEN_QTY' )
    PART_NO_Openso_3=fiilna(PART_NO_Openso_3, full_date_range , QTY_name= 'OPEN_QTY' )

    #計算兩種backlog變化率的計算
    Indicator= Foward_Indicator(PART_NO_Openso_1, PART_NO_Openso_2, PART_NO_Openso_3 )
    #判定條件是否為趨勢
    Trend_Indicator =Trend_Table(Indicator)

    temp=pd.DataFrame()
    temp['MATNR']=[PART_NO_name]
    temp['AI_TREND']=Trend_Indicator.loc[Trend_Indicator['Date'] == today_YM, 'labels'].values
    new_data = pd.concat([new_data, temp], ignore_index=True)

Commit_data= Upload_Data(new_data,Region_class,today)
#upload測試機/正式機
# Upload_Test_Environment(Commit_data)
# Upload_Deployment_Environment(Commit_data)
Commit_data

,MANDT,MATNR,WERKS,FCST_TYPE,FCST_DATE,CREATE_DATE,FCST_QTY,AIFCST_RATE,MOVING_AVG_QTY,MOVING_AVG_RATE,EVALUTE_FLAG,ALERT_FLAG,ZSTRATEGY,AI_TREND
0,168,PCM-24R2GL-AE,USH1,R,202501,20241102,0,0,0,0,0,0,,1
1,168,HPC-7442MB-00XE,USH1,R,202501,20241102,0,0,0,0,0,0,,0
2,168,MIC-75M13-00B1,USH1,R,202501,20241102,0,0,0,0,0,0,,0
3,168,PCE-5B13-03A1E,USH1,R,202501,20241102,0,0,0,0,0,0,,0
4,168,MIC-770H-20A1,USH1,R,202501,20241102,0,0,0,0,0,0,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,168,BB-9PCDT,USH1,R,202501,20241102,0,0,0,0,0,0,,0
260,168,TPC-61T-E3AE,USH1,R,202501,20241102,0,0,0,0,0,0,,0
261,168,EKI-2525M-BE,USH1,R,202501,20241102,0,0,0,0,0,0,,0
262,168,PCE-3B06-00A1E,USH1,R,202501,20241102,0,0,0,0,0,0,,0


In [28]:
Commit_data[Commit_data['AI_TREND']==1].reset_index(drop=True)

,MANDT,MATNR,WERKS,FCST_TYPE,FCST_DATE,CREATE_DATE,FCST_QTY,AIFCST_RATE,MOVING_AVG_QTY,MOVING_AVG_RATE,EVALUTE_FLAG,ALERT_FLAG,ZSTRATEGY,AI_TREND
0,168,PCM-24R2GL-AE,USH1,R,202501,20241102,0,0,0,0,0,0,,1
1,168,ADAM-4017+-F,USH1,R,202501,20241102,0,0,0,0,0,0,,1
2,168,OPT8H-AE,USH1,R,202501,20241102,0,0,0,0,0,0,,1
3,168,BB-485LDRC9,USH1,R,202501,20241102,0,0,0,0,0,0,,1
4,168,BB-USPTL4,USH1,R,202501,20241102,0,0,0,0,0,0,,1


In [29]:
Commit_data[Commit_data['AI_TREND']==-1].reset_index(drop=True)

,MANDT,MATNR,WERKS,FCST_TYPE,FCST_DATE,CREATE_DATE,FCST_QTY,AIFCST_RATE,MOVING_AVG_QTY,MOVING_AVG_RATE,EVALUTE_FLAG,ALERT_FLAG,ZSTRATEGY,AI_TREND
0,168,ADAM-6250-B,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
1,168,ADAM-6017-D,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
2,168,ADAM-6217-B,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
3,168,SQR-SD3I-4G1K6SNLB,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
4,168,MIC-75M20-00C1,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
5,168,PCE-5B13-08A2E,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
6,168,TPC-31T-E3AE,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
7,168,MIC-770W-20A1,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
8,168,EKI-2725-CE,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
9,168,SQF-S25M8-256G-SAE,USH1,R,202501,20241102,0,0,0,0,0,0,,-1
